# Hate speech detection - Transformers

In [ ]:
!pip install transformers
!pip install transformers[onnx]
!pip install datasets

In [2]:
import pandas as pd
import numpy as np
from scipy.special import softmax

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

import datasets 
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification

# Modelo 1: DistilBERT

El primer modelo utiliza DistilBERT, un modelo de transformers más pequeño y más rápido que BERT. Este modelo está entrenado utilizando el mismo corpus que BERT, con el objetivo de predecir las mismas probabilidades que el modelo base, haciendo uso de MLM (Masked Language Modeling):

In [3]:
checkpoint = "distilbert-base-uncased"

batch_size = 16
num_epochs = 3

El dataset utilizado es "Measuring Hate Speech", propio de HuggingFace:

In [4]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df.describe()

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-aea626f3e087c844


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-aea626f3e087c844/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,...,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,...,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,...,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,...,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,...,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,...,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


Se utilizará la columna `text`, que contiene caracteres de alfabetos no latinos:

In [5]:
df['text']

0         Yes indeed. She sort of reminds me of the elde...
1         The trans women reading this tweet right now i...
2         Question: These 4 broads who criticize America...
3         It is about time for all illegals to go back t...
4         For starters bend over the one in pink and kic...
                                ...                        
135551    عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي  نص...
135552    Millions of #Yemen-is participated in mass ral...
135553    @AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...
135554    Millions of #Yemen-is participated in mass ral...
135555    لا تتشمت الرجال مسكين يعاني كس امه 😂. يقول يال...
Name: text, Length: 135556, dtype: object

Por lo tanto, se filtra la columna para obtener únicamente los tweets escritos usando caracteres latinos:

In [6]:
df_latin = df[~df['text'].str.match(r'.*[^\x00-\xFF]')]
df_latin['text']

0         Yes indeed. She sort of reminds me of the elde...
1         The trans women reading this tweet right now i...
2         Question: These 4 broads who criticize America...
3         It is about time for all illegals to go back t...
4         For starters bend over the one in pink and kic...
                                ...                        
135536                        happy pride month i'm gay URL
135537                   I love that you enjoy being trans.
135538    Please help support nonbinary pride, add a #Tw...
135539                        Trans rights are human rights
135541                   I love that you enjoy being trans.
Name: text, Length: 124719, dtype: object

Se realiza una separación de 60% de los datos para entrenamiento, 20% para validación y 20% para pruebas. En total se tienen 74831 datos de entrenamiento, 24944 datos de validación y la misma cantidad de datos de prueba:

In [7]:
train_val_df, test_df = train_test_split(df_latin, test_size=0.2, random_state=9)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=9)

print(f'Train: {train_df.shape}\nValidation: {val_df.shape}\nTest: {test_df.shape}')

Train: (74831, 131)
Validation: (24944, 131)
Test: (24944, 131)


La variable objetivo es la columna `hatespeech`, una variable ordinal con tres niveles, donde 0 indica que el tweet no posee odio, y 2 indica que el tweet posee mucho odio:

In [8]:
train_df['hatespeech'].unique()

array([2., 0., 1.])

## Preprocesamiento

Inicialmente, para realizar el entrenamiento, se convierte la variable objetivo en binaria. En ese sentido, si el nivel de la variable `hatespeech` es mayor a 0, se añade a la clase positiva:

In [9]:
def hatespeech_to_binary(score):
    if score > 0:
        return 1
    return 0

Se define también la función `tokenize()`, que utiliza el tokenizer de DistilBERT para transformar un texto en una lista de tokens con el formato de entrada necesario para el modelo. Se utiliza el parámetro `truncation` para cortar aquellos textos que sean más largos que el tamaño máximo aceptado por el modelo:

In [10]:
def tokenize(example):
    return tokenizer(example['text'], truncation=True)

Posteriormente, se aplica la función `hatespeech_to_binary()` a la variable objetivo de los tres conjuntos de datos. Por ejemplo, se muestra que el conjunto de pruebas tiene 14816 datos de la clase "sin discurso de odio" y 10128 datos de la clase "discurso de odio":

In [13]:
train_df['hatespeech'] = train_df['hatespeech'].apply(hatespeech_to_binary)
val_df['hatespeech'] = val_df['hatespeech'].apply(hatespeech_to_binary)
test_df['hatespeech'] = test_df['hatespeech'].apply(hatespeech_to_binary)

pd.DataFrame({'train': train_df['hatespeech'].value_counts(), 'validation':val_df['hatespeech'].value_counts(), 'test':test_df['hatespeech'].value_counts()})

,train,validation,test
0,43895,14751,14816
1,30936,10193,10128


Se define el tokenizer de DistilBERT:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Y se define el objeto `data_collator` para crear los batches y aplicar padding dinámicamente a cada batch. Se retornan tensores para ser utilizados con `Tensorflow`: 

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Con los conjuntos de datos separados, se crean los conjuntos como objetos de la clase `Dataset` de HuggingFace:

In [ ]:
train_hf_dataset = datasets.Dataset.from_pandas(train_df)
val_hf_dataset = datasets.Dataset.from_pandas(val_df)
test_hf_dataset = datasets.Dataset.from_pandas(test_df)

Se aplica la tokenización utilizando el tokenizer de DistilBERT:

In [ ]:
tokenized_train = train_hf_dataset.map(tokenize, batched=True)
tokenized_val = val_hf_dataset.map(tokenize, batched=True)
tokenized_test = test_hf_dataset.map(tokenize, batched=True)

Parameter 'function'=<function tokenize at 0x7f8e656b2ef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/75 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

Y se convierten los Datasets resultantes en datasets de `Tensorflow`, especificando las columnas generadas `attention_mask` (para que el modelo ignore el padding) e `input_ids` (resultantes del mapeo palabra-token, es decir, el ID del token correspondiente) como variables de entrada, y `hatespeech` como la variable objetivo:

In [ ]:
tf_train_dataset = tokenized_train.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["hatespeech"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_val_dataset = tokenized_val.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["hatespeech"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_test_dataset = tokenized_test.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["hatespeech"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

## Fine-tuning

Para la ronda de fine-tuning, se define un objeto de la clase `PolynomialDecay` para reducir la tasa de aprendizaje durante el entrenamiento de manera lineal.

In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

Se descarga DistilBERT para realizar el entrenamiento:

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Se compila utilizando `SparseCategoricalCrossentropy` para calcular la pérdida con una variable objetivo representada con enteros. Además, se especifica que la salida del modelo no incluye la aplicación de la función sigmoide (con `from_logits=True`):

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=lr_scheduler),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

El modelo se puede observar a continuación:

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


Se entrena durante 3 épocas, pues se observa que la función de pérdida aumenta en el conjunto de validación después de cada época:

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=num_epochs)

Epoch 1/3
4677/4677 [==============================] - 963s 203ms/step - loss: 0.4164 - accuracy: 0.8119 - val_loss: 0.4020 - val_accuracy: 0.8190
Epoch 2/3
4677/4677 [==============================] - 951s 203ms/step - loss: 0.3669 - accuracy: 0.8363 - val_loss: 0.4093 - val_accuracy: 0.8160
Epoch 3/3
4677/4677 [==============================] - 951s 203ms/step - loss: 0.3151 - accuracy: 0.8579 - val_loss: 0.4593 - val_accuracy: 0.8079


## Predicciones

El conjunto de etiquetas de pruebas se obtiene con el dataset correspondiente:

In [ ]:
y_test = test_hf_dataset['hatespeech']
y_test

[1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,


Se realizan las predicciones sobre el conjunto de pruebas:

In [ ]:
pred = model.predict(tf_test_dataset, verbose=1)
pred

1559/1559 [==============================] - 100s 63ms/step


TFSequenceClassifierOutput([('logits', array([[-0.8292847 ,  0.813753  ],
                                    [-1.0089829 ,  1.0188779 ],
                                    [ 0.12207337, -0.14294875],
                                    ...,
                                    [ 2.413375  , -2.978281  ],
                                    [ 2.488507  , -3.0273027 ],
                                    [-0.22143112,  0.23835571]], dtype=float32))])

In [ ]:
y_pred = np.argmax(pred['logits'], axis=1)
y_pred

array([1, 1, 0, ..., 0, 0, 1])

Y se obtiene la matriz de confusión y las métricas de rendimiento:

In [ ]:
confusion_matrix(y_test, y_pred)

array([[12144,  2672],
       [ 2098,  8030]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.82      0.84     14816
           1       0.75      0.79      0.77     10128

    accuracy                           0.81     24944
   macro avg       0.80      0.81      0.80     24944
weighted avg       0.81      0.81      0.81     24944



## Persistencia del modelo

In [ ]:
model.save('transformer_model')

INFO:tensorflow:Assets written to: transformer_model/assets


INFO:tensorflow:Assets written to: transformer_model/assets


In [ ]:
!zip -r transformer_model.zip transformer_model

  adding: transformer_model/ (stored 0%)
  adding: transformer_model/variables/ (stored 0%)
  adding: transformer_model/variables/variables.index (deflated 79%)
  adding: transformer_model/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: transformer_model/assets/ (stored 0%)
  adding: transformer_model/keras_metadata.pb (deflated 94%)
  adding: transformer_model/saved_model.pb (deflated 93%)


In [ ]:
tokenizer.save_pretrained("local-tf-checkpoint")
model.save_pretrained("local-tf-checkpoint")

In [ ]:
!python -m transformers.onnx --model=local-tf-checkpoint onnx/

2022-07-25 01:12:08.836882: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-07-25 01:12:09.692659: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-07-25 01:12:11.120779: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.
All TF 2.0 model weights were used when initializing DistilBertModel.

All the weights of DistilBertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertModel for predictions without further training.
Using framework PyTorch: 1.12.0+cu113
/usr/local/lib/python3.7/dist-packages/transformers/models/distilbert/modeling_distilbert.py:214: TracerWarning: torch.te

# Modelo 2: Twitter-roBERTa (Preentrenado)

A modo de comparación, se utiliza un modelo ya entrenado sobre un conjunto de 58M de tweets: [roBERTa](https://huggingface.co/cardiffnlp/twitter-roberta-base-hate). Este modelo utiliza una función de preprocesamiento para eliminar las etiquetas de usuarios y los links:

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

Se obtiene el tokenizer desde HuggingFace:

In [ ]:
checkpoint_roberta = "cardiffnlp/twitter-roberta-base-hate"

In [ ]:
tokenizer_roberta = AutoTokenizer.from_pretrained(checkpoint_roberta)

Downloading:   0%|          | 0.00/588 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Y se utiliza para retornar un formato compatible con PyTorch:

In [ ]:
def tokenize_roberta(text):
    return tokenizer_roberta(text, return_tensors='pt')

Se obtiene el modelo:

In [ ]:
model_roberta = AutoModelForSequenceClassification.from_pretrained(checkpoint_roberta)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Y se aplica el preprocesamiento y la tokenización para el conjunto de pruebas:

In [ ]:
x_test_roberta = test_df['text'].apply(preprocess)
x_test_roberta

100928    ayo i even kill handicapped and crippled bitch...
66577     Deport every single illegal..period! Tax credi...
49307     Females that are lustful and sexually degenera...
56305     Lmao holy shit, I can just imagine you strokin...
35518     calling God's word retarded, lemme know how th...
                                ...                        
6418      I didn't know being Hispanic was something tha...
128232    oh that's cute. you didn't even read the thing...
26398     >Rameez Raja: West Indies fans bring a lot of ...
14253     Also, it's about celebrating the thousands and...
111893    They all need to be Kicked, taken out of our c...
Name: text, Length: 24944, dtype: object

In [ ]:
x_test_tokenized = x_test_roberta.apply(tokenize_roberta)

Se define una función para predecir un conjunto de tokens y retornar la probabilidad de que sea discurso de odio:

In [ ]:
def predict_roberta(text_tokenized):
    output = model_roberta(**text_tokenized)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

In [ ]:
pred_roberta = x_test_tokenized.apply(predict_roberta)
pred_roberta

100928      [0.3377411, 0.66225886]
66577      [0.20954299, 0.79045695]
49307       [0.12304736, 0.8769526]
56305      [0.84764236, 0.15235761]
35518      [0.9685684, 0.031431578]
                    ...            
6418      [0.97419703, 0.025802942]
128232       [0.4846025, 0.5153975]
26398      [0.9822295, 0.017770521]
14253     [0.98967487, 0.010325205]
111893      [0.14323045, 0.8567695]
Name: text, Length: 24944, dtype: object

In [ ]:
y_pred_roberta = [np.argmax(pred) for pred in pred_roberta]
y_pred_roberta

[1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,


Finalmente, se retorna la matriz de confusión y las métricas de rendimiento:

In [ ]:
confusion_matrix(test_df['hatespeech'], y_pred_roberta)

array([[11575,  3241],
       [ 5729,  4399]])

In [ ]:
print(classification_report(test_df['hatespeech'], y_pred_roberta))

              precision    recall  f1-score   support

           0       0.67      0.78      0.72     14816
           1       0.58      0.43      0.50     10128

    accuracy                           0.64     24944
   macro avg       0.62      0.61      0.61     24944
weighted avg       0.63      0.64      0.63     24944

